# HW 8 Naive Bayes

In [16]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

Part 1: Explain Laplace smoothing: 
A techinque used to address the issue of zero probabilities in categorical data, particularly when estimating probabilities from a limited set of observations. The problem arises when you are trying to estimate the probability of an event that has not been observed in your dataset. In such cases, traditional maximum likelihood estimation might assign a probability of zero, which can lead to problems when dealing with subsequent calculations, such as in the case of Bayesian inference.


Part II: Build a Na¨ıve Bayes algorithm on the titanic data set attached to OneDrive to
predict whether a passenger survived or not.

1. Import the data set into pandas data frame.


In [2]:
df=pd.read_csv("titanic.csv")
titanic=pd.DataFrame(df)
titanic.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


2. Split the data into training and test sets.

In [3]:
X = titanic.drop(columns=["Survived"])
y = titanic["Survived"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

3. Select one or more explanatory variables you would like to use.

In [5]:
explanatory_variables = titanic[["Age", "Sex", "Pclass", "Parch", "Fare", "Embarked"]]

4. Figure out if there are any missing values in the explanatory variables you want to use and either delete those passengers from the data set or fill in the missing values. If a numerical variable has missing values, you might fill those in with the average or median of that variable. If a categorical variable has missing values, you might fill those in using the most common value. You can create your own script for missing values or your can use sklearn SimpleImputer.

In [6]:
missing_values=explanatory_variables.isnull().sum()
print("missing values:\n", missing_values)

missing values:
 Age         177
Sex           0
Pclass        0
Parch         0
Fare          0
Embarked      2
dtype: int64


In [7]:
numerical_impute=SimpleImputer(strategy="mean")
explanatory_variables[["Age", "Fare"]] = numerical_impute.fit_transform(explanatory_variables[["Age", "Fare"]])

cat_impute=SimpleImputer(strategy="most_frequent")
explanatory_variables[["Sex", "Embarked"]] = cat_impute.fit_transform(explanatory_variables[["Sex", "Embarked"]])

after_imputation = explanatory_variables.isnull().sum()
print("\nMissing Values After Imputation:\n", after_imputation)


Missing Values After Imputation:
 Age         0
Sex         0
Pclass      0
Parch       0
Fare        0
Embarked    0
dtype: int64


C:\Users\User\AppData\Local\Temp\ipykernel_46524\2628404541.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  explanatory_variables[["Age", "Fare"]] = numerical_impute.fit_transform(explanatory_variables[["Age", "Fare"]])
C:\Users\User\AppData\Local\Temp\ipykernel_46524\2628404541.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  explanatory_variables[["Sex", "Embarked"]] = cat_impute.fit_transform(explanatory_variables[["Sex", "Embarked"]])


5. Convert the categorical variables to numerical using encoding. You can create your own script or use sklearn LabelEncoder.

In [8]:
print("Unique values before encoding:\n", explanatory_variables[["Sex", "Embarked"]])

Unique values before encoding:
         Sex Embarked
0      male        S
1    female        C
2    female        S
3    female        S
4      male        S
..      ...      ...
886    male        S
887  female        S
888  female        S
889    male        C
890    male        Q

[891 rows x 2 columns]


In [10]:
#?LabelEncoder
le=LabelEncoder()
explanatory_variables['Sex']=le.fit_transform(explanatory_variables['Sex'])
explanatory_variables['Embarked']=le.fit_transform(explanatory_variables['Embarked'])
#Quick Check
print("\nUnique values after encoding:\n", explanatory_variables[["Sex", "Embarked"]])


Unique values after encoding:
      Sex  Embarked
0      1         2
1      0         0
2      0         2
3      0         2
4      1         2
..   ...       ...
886    1         2
887    0         2
888    0         2
889    1         0
890    1         1

[891 rows x 2 columns]


C:\Users\User\AppData\Local\Temp\ipykernel_46524\3387429089.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  explanatory_variables['Sex']=le.fit_transform(explanatory_variables['Sex'])
C:\Users\User\AppData\Local\Temp\ipykernel_46524\3387429089.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  explanatory_variables['Embarked']=le.fit_transform(explanatory_variables['Embarked'])


6. Build a model on the training data. You can create your own code or use sklearn NaiveBayes. If you use a mix of continuous and categorical explanatory variables, think of how you can build the model. 

In [15]:
X_train, X_test, y_train, y_test = train_test_split(explanatory_variables, y, test_size=0.3, random_state=42)
NB=GaussianNB()
NB.fit(X_train,y_train)

GaussianNB()

7. Inspect the evaluation measures (accuracy score, confusion matrix, classification report).

In [17]:
y_pred=NB.predict(X_test)
#Accuracy Score
print("Accuracy:", accuracy_score(y_pred, y_test))

#Confusion Matrix
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

#Classification Report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7873134328358209

Confusion Matrix:
 [[131  26]
 [ 31  80]]

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.83      0.82       157
           1       0.75      0.72      0.74       111

    accuracy                           0.79       268
   macro avg       0.78      0.78      0.78       268
weighted avg       0.79      0.79      0.79       268



8. Take some values for the explanatory variables and use your model to predict if that person would have survived or not.

In [41]:
# Assume you have a set of values for explanatory variables
new_data = pd.DataFrame({
    "Age": [25],
    "Sex": ["female"],
    "Pclass": [2],
    "Parch": [0],
    "Fare": [30],
    "Embarked": ["S"]
})


new_data["Sex"] = le.fit_transform(new_data["Sex"])
new_data["Embarked"] = le.fit_transform(new_data["Embarked"])

cat_impute = SimpleImputer(strategy="most_frequent")
new_data[["Sex", "Embarked"]] = cat_impute.fit_transform(new_data[["Sex", "Embarked"]])

# Make predictions using the trained model
predictions = NB.predict(new_data)

# Interpret the predictions
if predictions[0] == 0:
    print("The person is predicted to not have survived.")
else:
    print("The person is predicted to have survived.")

print("This is similar to laplace smoothing in a sense, since the newdata has not occured in the titanic dataset.")

The person is predicted to have survived.
This is similar to laplace smoothing in a sense, since the newdata has not occured in the titanic dataset.
